In [ ]:
import json
import numpy as np

with open('knu/SentiWord_info.json', encoding='utf-8-sig', mode='r') as f:
    data = json.load(f)
    
data_list = [list(data[i].values()) for i in range(len(data))]
data_array = np.asarray(data_list)

word = np.char.strip(data_array[:,0])
polarity = data_array[:,2]

In [ ]:
def text2array(text):
    from konlpy.tag import Okt
    import numpy as np
    
    okt = Okt()
    lines = text.split(".")
    results = []
    for line in lines:
        wlist = okt.pos(line)
        r = []
        for w in wlist:
            if not w[1] in ['Josa', 'Eomi', 'Punctuation']:
                r.append(w[0])

        rl = (" ".join(r)).strip()
        results.append(rl)

    res = " ".join(results)
    res = res.split()
    comb = ["".join(res[m:n+1]) for m in range(len(res)) for n in range(m,m+2) if n < len(res)]
    comb = np.asarray(comb)
    
    return comb

In [ ]:
import os
import time
import pandas as pd
from pathos.multiprocessing import ProcessingPool as Pool

qlist = ["여성", "여자+-여성", "남성", '남자+-남성']

start = time.time()

version = "knu_score" #"new_dict_score"

for query in qlist:
    dir_ = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#3_"+version, "#3_"+query)
    if not os.path.exists(dir_):
        os.makedirs(dir_)

    for year in range(2000, 2020):
        for month in range(1, 13):
            xlsx_name1 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', "#2_html2text", "#2_" + query, '#2_merged_'+query, 
                                     f"{'#2_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            xlsx_name2 = os.path.join(os.path.dirname(os.getcwd()), 'xlsx_data', '#3_'+version, "#3_" + query,
                                     f"{'#3_'+str(year)+'_'+str(month).zfill(2)+'_'+query}.xlsx")

            df = pd.read_excel(xlsx_name1, index_col=0)
            SCORE = list()

            for i in range(0, len(df), 12):

                tlist = list()
                for j in range(12):
                    if i + j == len(df):
                        break
                    tlist.append(df.loc[i+j, 'text'])

                with Pool(12) as p:
                    clist = p.map(text2array, tlist)

                for comb in clist:
                    true_false = np.in1d(word, comb)
                    t_score = polarity[true_false]
                    if t_score.size:
                        score = sum(t_score.astype(int))/len(t_score)
                        SCORE.append(score)
                    else:
                        SCORE.append(0)

            df[version] = SCORE
            df.to_excel(xlsx_name2)

end = time.time()
print(end - start)